# Facial Keypoints Detection
- https://www.kaggle.com/c/facial-keypoints-detection/
  - 画像の回帰問題
  - 顔写真から目の位置をpixel単位で推定する
  - http://qiita.com/slowsingle/items/71fe9fb8efc033d7c562

In [1]:
!ls datasets/

training.csv


In [2]:
"""データセットの読込
"""

import pandas as pd

train_df = pd.read_csv("./datasets/training.csv").dropna()
train_df.head(1)

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...


In [3]:
import numpy as np
# from itertools import chain
# flatten = lambda l: list(chain.from_iterable(l))

# images = train_df.Image.apply(
#   lambda image: np.array([float(v) for v in image.split(' ')])
# ).apply(
# #   lambda image: flatten(image.reshape(1, 96, 96) / 255.0)
#   lambda image: image.reshape(1, 96, 96) / 255.0
# ).values

images = train_df.Image.apply(
#   lambda image: map(float, image.split(' ')).reshape(1, 96, 96) / 255.0
  lambda image: image.split(' ')
)
images.shape #.values.shape

(2140,)

In [4]:
targets = np.array([
    train_df.left_eye_center_x,
    train_df.left_eye_center_y,
    train_df.right_eye_center_x,
    train_df.right_eye_center_y,
  ]) / 96.0
targets = np.array(targets)

targets
# データ拡張は一旦やらない

array([[ 0.68784962,  0.67013475,  0.67767763, ...,  0.71282152,
         0.66825187,  0.69462245],
       [ 0.40627368,  0.36427163,  0.36364211, ...,  0.40262474,
         0.31970408,  0.35920238],
       [ 0.31486466,  0.31197163,  0.32191447, ...,  0.30099851,
         0.28125935,  0.32067177],
       [ 0.37939248,  0.34842411,  0.36364211, ...,  0.39184403,
         0.42570918,  0.40186395]])

In [6]:
"""モデル定義
"""

!pip install keras

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import MaxPooling2D, Conv2D
from keras.regularizers import l2

    100% |################################| 204kB 1.7MB/s 
    100% |################################| 3.1MB 243kB/s 
  Running setup.py bdist_wheel for keras ... - \ done
  Stored in directory: /root/.cache/pip/wheels/93/72/65/4924fd6b1859343291c50774e2df36919ee61c4511dc6a9890
  Running setup.py bdist_wheel for theano ... - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/d5/5b/93/433299b86e3e9b25f0f600e4e4ebf18e38eb7534ea518eba13
Successfully built keras theano


Using TensorFlow backend.


In [7]:
# model = Sequential()

# model.add(Convolution2D(8, 4, 4, border_mode='same', subsample=(2, 2), input_shape=(1, 96, 96)))
# model.add(Activation('relu'))
# model.add(Convolution2D(16, 4, 4, border_mode='same', subsample=(2, 2), input_shape=(8, 48, 48)))
# model.add(Activation('relu'))
# model.add(Convolution2D(32, 4, 4, border_mode='same', subsample=(2, 2), input_shape=(16, 24, 24)))
# model.add(Activation('relu'))

# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(1024, W_regularizer=l2(0.001)))
# model.add(Activation('relu'))
# model.add(Dense(4, W_regularizer=l2(0.001)))
# model.add(Activation('linear'))


model = Sequential()

model.add(Conv2D(8, (4, 4), padding='same', strides=(2, 2), input_shape=(1, 96, 96)))
model.add(Activation('relu'))
model.add(Conv2D(16, (4, 4), padding='same', strides=(2, 2), input_shape=(8, 48, 48)))
model.add(Activation('relu'))
model.add(Conv2D(32, (4, 4), padding='same', strides=(2, 2), input_shape=(16, 24, 24)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='th'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, W_regularizer=l2(0.001)))
model.add(Activation('relu'))
model.add(Dense(4, W_regularizer=l2(0.001)))
model.add(Activation('linear'))

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(images,
          targets, 
          batch_size=64, 
          nb_epoch=5, 
          validation_split=0.1,
          shuffle=True)
model.save_weights("regression_model_weights.hdf5")

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:27: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(data_format="channels_first", pool_size=(2, 2))`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python2.7/dist-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking model input: expected conv2d_1_input to have 4 dimensions, but got array with shape (2140, 1)

In [23]:
train_df = pd.read_csv("./datasets/training.csv").dropna()
# 把握しやすさを考慮する
images = list()
targets = list()
for i in train_df.index:
    image_rawdata = train_df.Image[i]
    image_vector = np.array(map(float, image_rawdata.split(' ')), dtype=np.float32)
    image = image_vector.reshape(1, 96, 96) / 255.0
    images.append(image)

    left_eye_center_x = train_df.left_eye_center_x[i]
    left_eye_center_y = train_df.left_eye_center_y[i]
    right_eye_center_x = train_df.right_eye_center_x[i]
    right_eye_center_y = train_df.right_eye_center_y[i]

    target = [left_eye_center_x, left_eye_center_y,
              right_eye_center_x, right_eye_center_y]

    targets.append(np.array(target) / 96.0)

#     # データ拡張
#     for j in range(10):
#         new_image = np.zeros(image.shape) + 0.5
#         if j == 0:
#             # 左にシフト
#             new_image[0, :, :-10] = image[0, :, 10:]
#             new_target = [left_eye_center_x - 10.0, left_eye_center_y,
#                           right_eye_center_x - 10.0, right_eye_center_y]
#         elif j == 1:
#             # 左にシフト
#             new_image[0, :, :-20] = image[0, :, 20:]
#             new_target = [left_eye_center_x - 20.0, left_eye_center_y,
#                           right_eye_center_x - 20.0, right_eye_center_y]
#         elif j == 2:
#             # 左にシフト
#             new_image[0, :, :-30] = image[0, :, 30:]
#             new_target = [left_eye_center_x - 30.0, left_eye_center_y,
#                           right_eye_center_x - 30.0, right_eye_center_y]
#         elif j == 3:
#             # 右にシフト
#             new_image[0, :, 10:] = image[0, :, :-10]
#             new_target = [left_eye_center_x + 10.0, left_eye_center_y,
#                           right_eye_center_x + 10.0, right_eye_center_y]
#         elif j == 4:
#             # 右にシフト
#             new_image[0, :, 20:] = image[0, :, :-20]
#             new_target = [left_eye_center_x + 20.0, left_eye_center_y,
#                           right_eye_center_x + 20.0, right_eye_center_y]
#         elif j == 5:
#             # 右にシフト
#             new_image[0, :, 30:] = image[0, :, :-30]
#             new_target = [left_eye_center_x + 30.0, left_eye_center_y,
#                           right_eye_center_x + 30.0, right_eye_center_y]
#         elif j == 6:
#             # やや上にシフト
#             new_image[0, :-10, :] = image[0, 10:, :]
#             new_target = [left_eye_center_x, left_eye_center_y - 10.0,
#                           right_eye_center_x, right_eye_center_y - 10.0]
#         elif j == 7:
#             # やや下にシフト
#             new_image[0, 10:, :] = image[0, :-10, :]
#             new_target = [left_eye_center_x, left_eye_center_y + 10.0,
#                           right_eye_center_x, right_eye_center_y + 10.0]
#         elif j == 8:
#             # かなり下にシフト
#             new_image[0, 20:, :] = image[0, :-20, :]
#             new_target = [left_eye_center_x, left_eye_center_y + 20.0,
#                           right_eye_center_x, right_eye_center_y + 20.0]
#         else:
#             # かなり下にシフト
#             new_image[0, 30:, :] = image[0, :-30, :]
#             new_target = [left_eye_center_x, left_eye_center_y + 30.0,
#                           right_eye_center_x, right_eye_center_y + 30.0]

#         images.append(new_image)
#         targets.append(np.array(new_target) / 96.0)

tmp_images, tmp_targets = np.array(images), np.array(targets)

In [31]:
images.shape

(2140,)

In [32]:
tmp_images.shape

(2140, 1, 96, 96)

In [39]:
tmp_images

array([[[[ 0.93333334,  0.9254902 ,  0.92941177, ...,  0.98039216,
           0.98039216,  0.98039216],
         [ 0.92156863,  0.93333334,  0.9254902 , ...,  0.97647059,
           0.98039216,  0.98431373],
         [ 0.92941177,  0.9254902 ,  0.92941177, ...,  0.98431373,
           0.98431373,  0.98039216],
         ..., 
         [ 0.72941178,  0.71764708,  0.70980394, ...,  0.20392157,
           0.22352941,  0.23529412],
         [ 0.74117649,  0.73725492,  0.81176472, ...,  0.23921569,
           0.27058825,  0.30588236],
         [ 0.74901962,  0.72156864,  0.72156864, ...,  0.27450982,
           0.29411766,  0.35294119]]],


       [[[ 0.85882354,  0.84313726,  0.80000001, ...,  0.36078432,
           0.34509805,  0.32941177],
         [ 0.87058824,  0.85882354,  0.86274511, ...,  0.36078432,
           0.34509805,  0.33725491],
         [ 0.90588236,  0.87843138,  0.83137256, ...,  0.3019608 ,
           0.3137255 ,  0.32941177],
         ..., 
         [ 0.00392157,  0.0039